In [2]:
# Load and preprocess
import pandas as pd
from sklearn.model_selection import train_test_split
from utils.preprocessing import load_and_clean_data

df = load_and_clean_data("data/raw/IMDB Dataset.csv")

# Split
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment'])

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Tokenization
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Encode labels for DL
y_train_dl = np.array(y_train)
y_test_dl = np.array(y_test)


In [4]:
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

lstm_model.summary()

lstm_model.fit(X_train_pad, y_train_dl, validation_split=0.2, epochs=5, batch_size=128)
lstm_model.evaluate(X_test_pad, y_test_dl)




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 64)           640000    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 673089 (2.57 MB)
Trainable params: 673089 (2.57 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5


250/250 [==============================] - 32s 120ms/step - loss: 0.4079 - accuracy: 0.8101 - val_loss: 0.3421 - val_accuracy: 0.8540
Epoch 2/5
250/250 [==============================] - 30s 118ms/step - loss: 0.2474 - accuracy: 0.9027 - val_loss: 0.2848 - val_accuracy:

[0.4046471118927002, 0.8694000244140625]

In [5]:
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

cnn_model.summary()

cnn_model.fit(X_train_pad, y_train_dl, validation_split=0.2, epochs=5, batch_size=128)
cnn_model.evaluate(X_test_pad, y_test_dl)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 64)           640000    
                                                                 
 conv1d (Conv1D)             (None, 196, 128)          41088     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 681217 (2.60 MB)
Trainable params: 681217 (2.60 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
250/250 [==============================] - 8s 28ms/step - loss: 0.4940 - a

[0.36207982897758484, 0.8705000281333923]

In [14]:
lstm_model.save("model/DL/lstm_model")
cnn_model.save("model/DL/cnn_model")

INFO:tensorflow:Assets written to: model/DL/lstm_model\assets


INFO:tensorflow:Assets written to: model/DL/lstm_model\assets


INFO:tensorflow:Assets written to: model/DL/cnn_model\assets


INFO:tensorflow:Assets written to: model/DL/cnn_model\assets


In [15]:
import os
from tensorflow.keras.models import load_model

# Load the saved models
lstm_model = load_model("model/DL/lstm_model")
cnn_model = load_model("model/DL/cnn_model")

# Print model summaries to see the architecture of each model
print("LSTM Model Summary:")
lstm_model.summary()

print("\nCNN Model Summary:")
cnn_model.summary()

# List files in the directory where the models are saved to ensure they are there
print("\nContents of the 'model/DL' directory:")
print(os.listdir("model/DL"))

LSTM Model Summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 64)           640000    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 673089 (2.57 MB)
Trainable params: 673089 (2.57 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

CNN Model Summary:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 64)           640000    
                        